# IIC-3670 NLP UC

- Versiones de librerías, python 3.8.10

- numpy 1.20.3
- flair 0.12
- allennlp 0.9.0

## Actividad en clase

Vamos a hacer fine tuning para NER tagger usando el tagger preentrenado de **flair**. Para esto haga lo siguiente:

- Importe el dataset CONLL_03_SPANISH.
- Cree el diccionario de labels a partir del corpus.
- Seleccione un pretrained BERT transformer encoder y cárguelo. Justifique su decisión. Revise la lista de modelos disponibles en https://huggingface.co/
- Use los embeddings para hacer fine tuning del tagger preentrenado de flair.
- Muestre un ejemplo de su ner funcionando.
- Cuanto termine, me avisa para entregarle una **L (logrado)**.
- Recuerde que las L otorgan un bono en la nota final de la asignatura.


***Tiene hasta el final de la clase.***

In [1]:
from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence
import flair.datasets

corpus = flair.datasets.CONLL_03_SPANISH()
print(corpus)

2024-04-17 13:41:59,455 Reading data from /home/marcelo/.flair/datasets/conll_03_spanish
2024-04-17 13:41:59,456 Train: /home/marcelo/.flair/datasets/conll_03_spanish/esp.train
2024-04-17 13:41:59,456 Dev: /home/marcelo/.flair/datasets/conll_03_spanish/esp.testa
2024-04-17 13:41:59,456 Test: /home/marcelo/.flair/datasets/conll_03_spanish/esp.testb
Corpus: 8323 train + 1915 dev + 1517 test sentences


In [2]:
label_dict = corpus.make_label_dictionary(label_type='ner', add_unk=False)
print(label_dict)

2024-04-17 13:42:05,677 Computing label dictionary. Progress:


8323it [00:00, 35528.27it/s]

2024-04-17 13:42:05,937 Dictionary created for label 'ner' with 4 values: ORG (seen 7390 times), LOC (seen 4914 times), PER (seen 4321 times), MISC (seen 2173 times)
Dictionary with 4 tags: ORG, LOC, PER, MISC


In [3]:
embeddings = TransformerWordEmbeddings(model='xlm-roberta-base',
                                       layers="-1",
                                       layer_mean=False,
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       model_max_length=512,
                                       )

In [4]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

tagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=label_dict,
    tag_type='ner',
    tag_format="BIOES",
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)


trainer = ModelTrainer(tagger, corpus)

2024-04-17 13:42:33,511 SequenceTagger predicts: Dictionary with 17 tags: O, S-ORG, B-ORG, E-ORG, I-ORG, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC


In [5]:
trainer.fine_tune(
    base_path="resources/taggers/ner-xlm-roberta-base",
    train_with_dev=False,
    max_epochs=8,
    learning_rate=2.0e-5,
    mini_batch_size=4,
    shuffle=False,
)


Gathered 16792 of total 250002
Reducing vocab size by 93.2833%
Reducing model size by 64.4163%
Reducing training parameter count by 64.4163%


2024-04-17 13:42:41,959 ----------------------------------------------------------------------------------------------------
2024-04-17 13:42:41,961 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250003, 768)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0): XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bi

2024-04-17 13:42:41,963 ----------------------------------------------------------------------------------------------------
2024-04-17 13:42:41,964 Corpus: "Corpus: 8323 train + 1915 dev + 1517 test sentences"
2024-04-17 13:42:41,965 ----------------------------------------------------------------------------------------------------
2024-04-17 13:42:41,965 Parameters:
2024-04-17 13:42:41,966  - learning_rate: "0.000020"
2024-04-17 13:42:41,966  - mini_batch_size: "4"
2024-04-17 13:42:41,967  - patience: "3"
2024-04-17 13:42:41,969  - anneal_factor: "0.5"
2024-04-17 13:42:41,970  - max_epochs: "8"
2024-04-17 13:42:41,971  - shuffle: "False"
2024-04-17 13:42:41,973  - train_with_dev: "False"
2024-04-17 13:42:41,974  - batch_growth_annealing: "False"
2024-04-17 13:42:41,975 ----------------------------------------------------------------------------------------------------
2024-04-17 13:42:41,976 Model training base path: "resources/taggers/ner-xlm-roberta-base"
2024-04-17 13:42:41,977 -

100%|██████████| 479/479 [00:12<00:00, 39.65it/s]

2024-04-17 13:46:25,958 Evaluating as a multi-label problem: False


...

2024-04-17 14:12:36,776 Evaluating as a multi-label problem: False
2024-04-17 14:12:36,843 DEV : loss 0.16008912026882172 - f1-score (micro avg)  0.885
2024-04-17 14:12:39,027 ----------------------------------------------------------------------------------------------------
2024-04-17 14:12:39,029 Testing using last state of model ...


100%|██████████| 380/380 [00:09<00:00, 38.52it/s]

2024-04-17 14:12:48,916 Evaluating as a multi-label problem: False


2024-04-17 14:12:48,957 0.8891	0.9036	0.8963	0.8606
2024-04-17 14:12:48,958 
Results:
- F-score (micro) 0.8963
- F-score (macro) 0.8886
- Accuracy 0.8606

By class:
              precision    recall  f1-score   support

         ORG     0.8640    0.9164    0.8894      1400
         LOC     0.8930    0.8625    0.8775      1084
         PER     0.9666    0.9837    0.9751       735
        MISC     0.8160    0.8088    0.8124       340

   micro avg     0.8891    0.9036    0.8963      3559
   macro avg     0.8849    0.8929    0.8886      3559
weighted avg     0.8894    0.9036    0.8961      3559

2024-04-17 14:12:48,958 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8963210702341137,
 'dev_score_history': [0.8251685393258427,
  0.8822727272727271,
  0.8860615699193457,
  0.8966461327857631,
  0.8978519195612431,
  0.8915110551036775,
  0.8934707903780069,
  0.8850245967280632],
 'train_loss_history': [0.49164654559153087,
  0.09582007639815401,
  0.08102016101342534,
  0.0659440910564966,
  0.04863045349449287,
  0.03785637262144272,
  0.030741337178146148,
  0.029474600166816564],
 'dev_loss_history': [0.11206331849098206,
  0.1204131469130516,
  0.12456652522087097,
  0.1320403665304184,
  0.13759379088878632,
  0.14882870018482208,
  0.16359102725982666,
  0.16008912026882172]}

In [6]:
# load the model you trained
model = SequenceTagger.load("resources/taggers/ner-xlm-roberta-base/final-model.pt")

# create example sentence
from flair.data import Sentence
sentence = Sentence("George Washington fue a Washington")

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2024-04-17 14:13:16,297 SequenceTagger predicts: Dictionary with 17 tags: O, S-ORG, B-ORG, E-ORG, I-ORG, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC
Sentence[5]: "George Washington fue a Washington" → ["George Washington"/PER, "Washington"/LOC]
